# Project 3: Solutions


In project 2, we conducted an exploratory analysis on the Admissions dataset. In this project, we will now build a model to predict whether someone is admitted to grad school based on their GRE score, undergrad GPA, and the prestige of their undergrad school.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import neighbors
from sklearn import grid_search
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%matplotlib inline

## Part 1. Load Dataset

#### 1.1 Let's load the dataset and check the first five rows

In [ ]:
# load data
df = pd.read_csv("../assets/admissions.csv")

# check head
df.head()

## Part 2. Pre-Process Data

#### 2.1 Check and remove missing values
**Reading**: Read Pandas docs on handling missing values:
[http://pandas.pydata.org/pandas-docs/stable/missing_data.html](http://pandas.pydata.org/pandas-docs/stable/missing_data.html)

In [ ]:
# check for missing values in each column before dropping
print "Missing values:"
print df.isnull().sum()

In [ ]:
# drop missing values if there are any
if df.isnull().sum().sum():
    print "There are missing values"
    df = df.dropna()
    print "Missing values dropped"

In [ ]:
# check for missing values in each column after dropping
print "Missing values:"
print df.isnull().sum()

#### 2.2 Check and convert all data types to numerical

In [ ]:
# check data types
print df.dtypes

In [ ]:
# Get numerical columns
num_cols = set(df.dtypes[((df.dtypes=="int64")|(df.dtypes=="float64"))].index)
non_cols = set(df.columns)-num_cols

print "Numerical columns:"
print num_cols
print "Non-numerical columns:"
print non_cols

# here all columns are numeric; no need to convert

#### 2.3 Check and create dummy variables for categorical features
**Reading**: API Docs for `get_dummies()`:
[http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)

In [ ]:
# create dummy variables for `prestige` feature
# this check allows this code to be run multiple times
if 'prestige' in df.columns:
    # get dummy variables for prestige
    df = df.join(pd.get_dummies(df['prestige'], prefix='prestige'))
    # remove prestige column
    df.drop(['prestige'], axis=1, inplace=True)

In [ ]:
# check for newly added columns
df.head()

## Part 3. Cross Validation

#### 3.1 Create separate training and test sets
**Reading**:

Read Scikit docs on cross validation:
[http://scikit-learn.org/stable/modules/cross_validation.html](http://scikit-learn.org/stable/modules/cross_validation.html)

Read Scikit docs on `sklearn.cross_validation.train_test_split()`:
[http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html)

In [ ]:
# set X and y
X = df.drop(['admit'], axis=1)
y = df['admit']

In [ ]:
# create separate training and test set with 60/40 train/test split
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
# check size of training set
print X_train.shape, y_train.shape

# check size of test set
print X_test.shape, y_test.shape

## Part 4. Build Model

#### 4.1 Build KNN Classifier
**Reading**: Read Scikit docs for `sklearn.neighbors.KNeighborsClassifier`:
[http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier)

In [ ]:
# instantiate knn classifier using default params
knn = neighbors.KNeighborsClassifier()

# train knn classifier on training set
knn.fit(X_train, y_train)

#### 4.2 Evaluate Model
**Reading**: Read Scikit docs on evaluating models: [http://scikit-learn.org/stable/modules/model_evaluation.html](http://scikit-learn.org/stable/modules/model_evaluation.html)

In [ ]:
# check model accuracy on test set
accuracy = knn.score(X_test, y_test)
print "Accuracy: " + str(accuracy)

**Reading**: Read Scikit docs on confusion matrix: [http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
# Get confusion matrix on test set
y_pred = knn.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

ax = plt.axes()
sns.heatmap(cm_normalized, annot=True)
ax.set_ylabel('True')
ax.set_xlabel('Pred')
plt.show()

print cm

## Part 5. Tune Model

#### 5.1 Perform Grid Search for `n_neighbors`
**Reading**: Read Scikit docs for `sklearn.grid_search.GridSearchCV`:
[http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html)

In [ ]:
# set list of values to grid search over
k = range(2, 25)
params = {'n_neighbors': k}

# perform grid search using list of values
gs = grid_search.GridSearchCV(
    estimator=neighbors.KNeighborsClassifier(),
    param_grid=params)
gs.fit(X_train, y_train)

# get best value to use
print gs.best_estimator_

In [ ]:
# plot search values vs. grid scores
plt.plot(k, [s[1] for s in gs.grid_scores_])

#### 5.2 Update model using best `n_neighbors`

In [ ]:
# instantiate knn classifier using updated params
knn = neighbors.KNeighborsClassifier(n_neighbors=2)

# train updated knn classifier on training set
knn.fit(X_train, y_train)

#### 5.3 Evaluate updated model

In [ ]:
# check updated model accuracy on test set
accuracy = knn.score(X_test, y_test)
print "Accuracy: " + str(accuracy)

In [ ]:
# Get confusion matrix on test set
y_pred = knn.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

ax = plt.axes()
sns.heatmap(cm_normalized, annot=True)
ax.set_ylabel('True')
ax.set_xlabel('Pred')
plt.show()

print cm

**Note** This is not a good model. Perhaps the data set is too small or the features themselves do not have enough predictive capability. However, the steps taken in this project are valid and applicable to future classifications problems.